# Teste e prototipagem de parseador de CSV do formulário

In [20]:
import pandas as pd
import numpy as np
import json
import datetime as dt 

#import xavy.test as xx
import xavy.dataframes as xd

import clean as cc

* Ver se surgem nulls em datas preenchidas quando as padronizamos. Isso indica formatação estranha.

## Functions

## Translations

In [21]:
# Load translations:
translation_df = pd.read_excel('../docs/conteudo_site/textos_site_traducao_traduzidos_v02.xlsx', sheet_name='Site')

# Build translators:
tema_map = cc.get_translator(translation_df, r'filtro_tema_\d')
tipo_map = cc.get_translator(translation_df, r'filtro_tipo_\d', lower=True)
pais_map = cc.get_translator(translation_df, r'filtro_pais_\d')

## Loading data

In [24]:
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230825081146_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230904055149_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230914064259_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230920091624_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231025093912_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231025104350_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231027103427_dados-abertos.csv', sep=';')
raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231027071604_dados-abertos.csv', sep=';')

## Cleaning and preparing the data

In [26]:
# Hard-coded parameters:
max_datasets = 5

# Start cleaning process by copying data and creating an ID:
cleaned_df = raw_df.copy()
hash_ids = cc.build_hash_id(raw_df)
cleaned_df.insert(0, 'hash_id', hash_ids)

# Change NaN for None:
cleaned_df = cleaned_df.replace({np.nan: None})

# Standardize used dataset column names:
cc.rename_first_data_cols(cleaned_df)

# Basic cleaning of strings:
str_cols = ['name', 'url', 'description', 'authors', 'countries', 'email', 'tags', 'url_source', 'url_image', 'comment']
for c in str_cols:
    cleaned_df[c] = cleaned_df[c].str.strip()
    
# Basic cleaning of strings associated to datasets:
data_str_cols = ['data_name', 'data_institution', 'data_url']
for i in range(1, max_datasets + 1):
    for c in data_str_cols:
        col = '{}_{}'.format(c, i)
        cleaned_df[col] = cleaned_df[col].str.strip()

# Remove empty URLs (e.g. 'https://'):
url_cols = xd.sel_col_by_regex(cleaned_df, 'url')
#cleaned_df.loc[:, url_cols] = cleaned_df.loc[:, url_cols].replace('^h?t?t?p?s?:?/?/?$', None, regex=True)

# Lowercase tags:
cleaned_df['tags'] = cleaned_df['tags'].str.lower()

# Split terms by semicolons:
semicolon_cols = ['authors', 'email', 'tags']
for c in semicolon_cols:
    cleaned_df[c] = cc.split_semicolons(cleaned_df[c])
    
# Split terms by colons:
colon_cols = ['countries']
for c in colon_cols:
    cleaned_df[c] = cc.split_semicolons(cleaned_df[c], ',')

# Parse usecase types and topics covered:
assert set(cc.type_names)  == set(xd.sel_col_by_regex(cleaned_df, '^type_'))
assert set(cc.topic_names) == set(xd.sel_col_by_regex(cleaned_df, '^topics_'))
cleaned_df['type']   = cc.options_to_list(cleaned_df, cc.type_names)
cleaned_df['topics'] = cc.options_to_list(cleaned_df, cc.topic_names)

# Parse datasets used by the usecases:
data_cols = xd.sel_col_by_regex(cleaned_df, '^data_')
data_df  = cleaned_df[data_cols]
cleaned_df['datasets'] = data_df.apply(lambda row: cc.data_info_dict_list(cc.series2transposed_df(row), max_datasets), axis=1)

# Change NaN for None:
cleaned_df = cleaned_df.replace({np.nan: None})

# Add processing date:
cleaned_df['record_date'] = dt.datetime.today().strftime('%Y-%m-%d')

# Update image URL:
cleaned_df['url_image'] = 'https://raw.githubusercontent.com/cewebbr/cordata/main/imagens/hash_id_' + cleaned_df['hash_id'].astype(str) + '.png'

# Select relevant columns:
trash_cols = xd.sel_col_by_regex(cleaned_df, r'type_|topics_|data_|continuar|_repeticao')
good_cols  = [col for col in cleaned_df.columns if col not in trash_cols]
cleaned_json = cleaned_df[good_cols].to_dict(orient='records')

In [28]:
# Add metadata (last update):
metadata = {'last_update': dt.datetime.today().strftime('%Y-%m-%d')}
wmeta_json = cc.embed_metadata(cleaned_json, metadata)

In [30]:
# Load previously saved data:
if True:
    with open('../dados/limpos/usecases_n11_2023-10-27.json', 'r') as f:
        previous_json = json.load(f)

In [31]:
# Join old data with new data:
updated_json = {'metadata': wmeta_json['metadata'], 
                'data': previous_json['data'] + wmeta_json['data']}

In [32]:
# Add translation:
translated_data = [cc.add_translation(case_json, tipo_map, tema_map, pais_map) for case_json in updated_json['data']]

# Join old data with new data:
updated_json = {'metadata': updated_json['metadata'], 
                'data': translated_data}

In [33]:
# Save to file:
if False:
    with open('../dados/limpos/usecases_n12_2023-10-27.json', 'w') as f:
        json.dump(updated_json, f, ensure_ascii=False, indent=2)

In [11]:
cleaned_df['data_name_3']

0    None
Name: data_name_3, dtype: object

In [18]:
updated_json['data'][0]

{'hash_id': 2507397523,
 'name': 'Três partidos do Centrão controlam mais de R$ 149,6 bilhões do governo Bolsonaro',
 'url': 'https://oglobo.globo.com/politica/tres-partidos-do-centrao-controlam-mais-de-1496-bilhoes-do-governo-bolsonaro-1-25365038',
 'description': 'Matéria jornalística publicada no Jornal O Globo que aponta o número de postos ocupados por integrantes e indicados dos partidos PP, PL e Republicanos na administração federal, juntamente com uma estimativa do montante de recursos direcionados aos respectivos órgãos pelo orçamento federal.',
 'pub_date': '01/2022',
 'authors': ['Dimitrius Dantas', 'Daniel Gullino', 'Bruno Góes'],
 'countries': ['Brasil'],
 'email': ['bruno.goes@oglobo.com.br', 'daniel.gullino@bsb.oglobo.com.br'],
 'tags': ['orçamento federal',
  'orçamento secreto',
  'bolsonaro',
  'centrão',
  'fisiologismo',
  'presidencialismo de coalizão'],
 'url_source': 'https://github.com/gabinete-compartilhado-acredito/dados_orcamentarios',
 'url_image': 'https://r

## Enriching data

In [37]:
institutions = set(cleaned_df[xd.sel_col_by_regex(cleaned_df, 'data_institution_')].values.flatten())
institutions

{'CGU',
 'Câmara dos deputados',
 'Ministério da Economia',
 None,
 'Prefeitura do Município de São Paulo',
 'SEADE',
 'Senado Federal',
 'TSE'}

## Visualizing cleaned data

### JSON

In [129]:
updated_json['data'][3]

{'hash_id': 1191967625,
 'name': 'Tutorial de dados',
 'url': 'https://github.com/tecMTST/tutorial-de-dados',
 'description': 'Pequenos projetos de análise de dados desenvolvidos em notebooks de Python que podem ser seguidos como tutoriais.',
 'pub_date': '09/2023',
 'authors': ['Núcleo de Tecnologia do MTST'],
 'countries': ['Brasil'],
 'email': ['mtst@nucleodetecnologia.com.br'],
 'tags': ['tutorial',
  'análise de dados',
  'ciência de dados',
  'expectativa de vida',
  'financiamento de campanha',
  'financiamento eleitoral',
  'desigualdade',
  'são paulo'],
 'url_source': 'https://github.com/tecMTST/tutorial-de-dados',
 'url_image': 'https://nucleodetecnologia.com.br/assets/img/novo-logo-tecnologia.svg',
 'comment': None,
 'type': ['material didático', 'estudo independente'],
 'topics': ['Direitos Humanos',
  'Imóveis, Habitação e Urbanismo',
  'Política, Partidos e Eleições',
  'Saúde'],
 'datasets': [{'data_name': 'Mortalidade no município de Sâo Paulo',
   'data_institution': 

### CSV

In [8]:
s_cols = 9
n_cols = cleaned_df.shape[-1]
for i in range(n_cols // s_cols + 1):
    display(cleaned_df.iloc[:, i * s_cols:(i + 1) * s_cols])

,name,url,description,pub_date,authors,countries,email,type_artigo,type_materia_jornalistica
0,Três partidos do Centrão controlam mais de R$ ...,https://oglobo.globo.com/politica/tres-partido...,Matéria jornalística publicada no Jornal O Glo...,01/2022,"[Dimitrius Dantas, Daniel Gullino, Bruno Góes]",[Brasil],"[bruno.goes@oglobo.com.br, daniel.gullino@bsb....",0,1
1,Serenata de amor,https://serenata.ai,Um projeto aberto que usa ciência de dados com...,06/2016,"[Open Knowledge Brasil, Irio Musskopf]",[Brasil],[contato@serenata.ai],0,0
2,Base dos Dados,https://basedosdados.org/,Catálogo de conjuntos de dados com alguns conj...,09/2020,[Instituto Base dos Dados],"[Mundial, Brasil]",NaN,0,0
3,Tutorial de dados,https://github.com/tecMTST/tutorial-de-dados,Pequenos projetos de análise de dados desenvol...,09/2023,[Núcleo de Tecnologia do MTST],[Brasil],[mtst@nucleodetecnologia.com.br],0,0


,type_aplicativo_plataforma,type_painel,type_conjunto_de_dados,type_material,type_estudo,type_bot,type_outro,topics_agropecuaria,topics_alimentacao
0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,1,1
3,0,0,0,1,1,0,0,0,0


,topics_administracao_publica,topics_arte_e_cultura,topics_atividade_industrial_de_comercio_ou_servicos,topics_ciencia_e_tecnologia,topics_comunicacoes,topics_consumo,topics_dados_demograficos,topics_defesa,topics_direito_e_processual_penal
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0


,topics_direitos_humanos,topics_economia,topics_educacao,topics_energia,topics_esporte,topics_financas_e_orcamento_publico,topics_imoveis_habitacao_e_urbanismo,topics_justica_e_direito,topics_lazer
0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0
2,1,1,1,1,1,1,1,1,1
3,1,0,0,0,0,0,1,0,0


,topics_meio_ambiente,topics_multimidia,topics_politica_partidos_e_eleicoes,topics_previdencia_e_assistencia_social,topics_recursos_hidricos,topics_recursos_minerais,topics_redes_sociais,topics_relacoes_internacionais,topics_religiao
0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1
3,0,0,1,0,0,0,0,0,0


,topics_saude,topics_seguranca,topics_terras,topics_transporte,topics_transparencia,topics_trabalho_e_emprego,topics_turismo,tags,url_source
0,0,0,0,0,0,0,0,"[orçamento federal, orçamento secreto, bolsona...",https://github.com/gabinete-compartilhado-acre...
1,0,0,0,0,1,0,0,"[ceap, Cota para Exercício da Atividade Parlam...",https://github.com/okfn-brasil/serenata-de-amor
2,1,1,1,1,1,1,1,"[catálogo, repositório]",https://github.com/basedosdados
3,1,0,0,0,0,0,0,"[tutorial, análise de dados, ciência de dados,...",https://github.com/tecMTST/tutorial-de-dados


,url_image,data_name_1,data_institution_1,data_url_1,data_periodical_1,continuar,data_name_2,data_institution_2,data_url_2
0,https://ogimg.infoglobo.com.br/in/25354165-974...,SIGA Brasil,Senado Federal,https://www12.senado.leg.br/orcamento/sigabrasil,2,1,Orçamento da despesa pública,CGU,https://portaldatransparencia.gov.br/orcamento...
1,https://d33wubrfki0l68.cloudfront.net/44a83a26...,Cota parlamentar,Câmara dos deputados,https://www.camara.leg.br/cota-parlamentar/,1,1,Informações sobre deputados federais,Câmara dos deputados,https://www.camara.leg.br/SitCamaraWS/Deputado...
2,https://avatars.githubusercontent.com/u/710976...,NaN,NaN,NaN,1,2,NaN,NaN,NaN
3,https://nucleodetecnologia.com.br/assets/img/n...,Mortalidade no município de Sâo Paulo,Prefeitura do Município de São Paulo,https://www.prefeitura.sp.gov.br/cidade/secret...,2,1,"População do município de São Paulo, por idade",SEADE,https://repositorio.seade.gov.br/group/seade-p...


,data_periodical_2,segunda_repeticao,data_name_3,data_institution_3,data_url_3,data_periodical_3,terceira_repeticao,data_name_4,data_institution_4
0,2.0,1.0,Plataforma +Brasil,Ministério da Economia,https://antigo.plataformamaisbrasil.gov.br/dow...,2.0,2.0,NaN,NaN
1,1.0,1.0,API de Compras Governamentais,Ministério da Economia,http://compras.dados.gov.br,1.0,2.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,1.0,Fronteiras dos distritos do município de São P...,Prefeitura do Município de São Paulo,https://geosampa.prefeitura.sp.gov.br,2.0,1.0,Prestação de contas eleitorais,TSE


,data_url_4,data_periodical_4,quarta_repeticao,data_name_5,data_institution_5,data_url_5,data_periodical_5,comment,type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[matéria jornalística]
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[painel, dashboard ou infográfico, bot]"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"São muitas bases de dados, não seria possível ...",[conjunto de dados]
3,https://dadosabertos.tse.jus.br/dataset/,2.0,1.0,Candidatos eleitorais,TSE,https://dadosabertos.tse.jus.br/dataset/,2.0,NaN,"[material didático, estudo independente]"


,topics,datasets
0,"[Administração Pública, Finanças e Orçamento P...","[{'data_name': 'SIGA Brasil', 'data_institutio..."
1,"[Administração Pública, Finanças e Orçamento P...","[{'data_name': 'Cota parlamentar', 'data_insti..."
2,"[Agropecuária, Alimentação, Administração Públ...",[]
3,"[Direitos Humanos, Imóveis, Habitação e Urbani...",[{'data_name': 'Mortalidade no município de Sâ...


## Testando funções

In [28]:
good_series = pd.Series(['1970-01', '1970-12', '1970-9', '2099-1', '2099-10', 
                         '01/1970', '12/1970', '9/1970', '1/2099', '10/2099',
                         ' 1970-01', '1970-12 ', ' 1970-9 ', '  2099-1  ', '2099-10', 
                         '01/1970 ', '  12/1970', '9/1970 ', '  1/2099', '10/2099  ', 
                         np.NaN, 'janeiro de 2020', '13/2000', '1800-01', '1900-01'])
good_answers = pd.Series(['1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10',
                         np.NaN, np.NaN, np.NaN, np.NaN, '1900-01'])

xx.multi_test_function(std_date_series, [(good_series,)], [good_answers])

In [13]:
good_series  = pd.Series(['http://cordata.nic.br', 'https://x.com/v2/api', 'ftp://henriquexavier.net', 'did:google.com?q=abacate&t=39', 'http://www.ftp.f/aba#dsh/fvuih487&#%$@$#.html', np.NaN,
                          'htp://www.globo.com', 'www.gov.br', 'http://www efd.com', 'http://gov.br/index.html ', '   http://gov.br/index.html ', ''])
good_answers = pd.Series([False, False, False, False, False, False, True, True, True, False, False, False])

xx.multi_test_function(bad_url, [(good_series,)], [good_answers])